In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
tf.keras.backend.set_floatx('float64')
from misc import *
from collections import deque
from nets import Q1, Actor
import random


import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
tf.keras.backend.set_floatx('float64')
from misc import *
from collections import deque
from nets import Q1, Actor, Q2
import random

In [19]:

class Q2(tf.keras.Model):
    def __init__(self):
        super(Q2,self).__init__()

        self.l1 = Dense(10, input_shape=(2,),kernel_initializer=tf.random_uniform_initializer(),
        bias_initializer = tf.random_uniform_initializer(),
        kernel_regularizer=tf.keras.regularizers.l1(0.01),
    activity_regularizer=tf.keras.regularizers.l2(0.01))

        self.l2 = Dense(33, kernel_regularizer=tf.keras.regularizers.l1(0.01),
    activity_regularizer=tf.keras.regularizers.l2(0.01),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())
        self.l3 = Dense(33, kernel_regularizer=tf.keras.regularizers.l1(0.01),
    activity_regularizer=tf.keras.regularizers.l2(0.01),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())
        self.l5 = Dense(2, kernel_regularizer=tf.keras.regularizers.l1(0.01),
    activity_regularizer=tf.keras.regularizers.l2(0.01),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())

    def call(self, input):
        feat = tf.nn.relu(self.l1(input))
        feat = tf.nn.dropout(feat, rate=0.01)
        value = tf.nn.sigmoid(self.l5(feat))
        return value

    def predict(self, history_one):
        inp = np.expand_dims(history_one, axis=0)
        return np.squeeze(self(inp).numpy())

    def initialize(self):
        inp = np.expand_dims(np.array([[0.],[0.]]),axis=1)
        self(inp)
        return

    def __str__(self):
        return self.name


In [20]:

def real_training(run_id, lr = 10**-2,
 states_wasted=10**3, batch_size=64, epochs=10, buffer_size=10**3, predict_q_table=True):

    q1=Q1()
    q2=Q2()
    actor = Actor()

    optimizer_critic = tf.keras.optimizers.Adam(lr = lr)
    optimizer_actor = tf.keras.optimizers.Adam(lr = lr)
    optimizer_l2 = tf.keras.optimizers.Adam(lr = lr)

    pt=[]
    rt=[]
    rts = []
    loss_ev = []
    history_betas = []
    history_would_have_done=[]

    betas_test = np.arange(-1,0,.05)
    optimal = max(ps_maxlik(betas_test))
    buffer = ReplayBufferKennedy(buffer_size=buffer_size)

    for episode in tqdm(range(states_wasted)):

        phase = np.random.choice([-1,1])

        beta_would_do = np.squeeze(actor.give_action().numpy())
        g=[]
        for n1 in [0,1]:
            g.append([-1,1][np.argmax(np.squeeze(q2( np.expand_dims(np.array([[beta_would_do, n1]]) ,axis=1 )).numpy()))])

        beta = beta_would_do + np.random.uniform(-.5,.5)
        history_betas.append(beta)
        history_would_have_done.append(beta_would_do)

        outcome = give_outcome(phase*0.4, beta)
        if np.random.random()<0.1:
            guess = np.random.choice([-1,1],1)[0]
        else:
            predictiones_guess = np.squeeze(q2( np.expand_dims(np.array([[beta, outcome]]) ,axis=1 )).numpy())
            guess = [-1,1][np.argmax(predictiones_guess)]

        pt.append(ps(beta_would_do,g))
        if guess == phase:
            reward=1
        else:
            reward = 0

        rt.append(reward)
        rts.append(np.sum(rt))
        buffer.add(beta, outcome, guess, reward)

        actions_did, outcomes, guessed, rewarded = buffer.sample(batch_size)

        histories = np.array([[be, out] for be, out in zip(actions_did, outcomes)])

        if episode%10 ==0:
            print("******")
            print("******")

            betass = np.arange(-1,0,.1)

            shistory = []
            for b in betass:
                shistory.append([[b, 0.]])
                spred_guess_0 = np.squeeze(q2(np.expand_dims(np.array(shistory), axis=1)).numpy())
            print(spred_guess_0)

        if episode >0:

            with tf.GradientTape() as tape:
                tape.watch(q2.trainable_variables)
                predictions_l2 = q2(np.expand_dims( histories, axis=1))
                #loss=tf.keras.losses.binary_crossentropy(np.expand_dims(rewarded,axis=1), predictions_l2)
                loss = tf.keras.losses.MSE(np.expand_dims(rewarded,axis=1), predictions_l2)
                loss = tf.reduce_mean(loss)
                grads = tape.gradient(loss, q2.trainable_variables)
                optimizer_l2.apply_gradients(zip(grads, q2.trainable_variables))

            with tf.GradientTape() as tape:
                tape.watch(q1.trainable_variables)
                predictions = q1(np.expand_dims(np.array(actions_did),axis=1))
                
                next_predictions= np.squeeze(predictions_l2.numpy())
                print(next_predictions)
                next_greedy_predictions=[]
                for k in range(len(next_predictions)):
                    next_greedy_predictions.append(np.max(next_predictions[k]))
                print(next_greedy_predictions)
                print("          ")
                print("chechehcehchehchhcehceh")

                loss_sum = tf.keras.losses.MSE(np.expand_dims(np.array(next_greedy_predictions),axis=1),predictions)
                loss = tf.reduce_mean(loss_sum)
                grads = tape.gradient(loss, q1.trainable_variables)
                optimizer_critic.apply_gradients(zip(grads, q1.trainable_variables))
                loss_ev.append(np.squeeze(loss.numpy()))
    #
            with tf.GradientTape() as tape:
                actions = actor(np.expand_dims(np.zeros(batch_size),axis=1))
                tape.watch(actions)
                qvals = q1(actions)
            dq_da = tape.gradient(qvals, actions)

            with tf.GradientTape() as tape:
                tape.watch(actor.trainable_variables)
                actions = actor(np.expand_dims(np.zeros(batch_size),axis=1))
                theta = actor.trainable_variables
            da_dtheta = tape.gradient(actions, theta, output_gradients=-dq_da)
            optimizer_actor.apply_gradients(zip(da_dtheta, actor.trainable_variables))

    rtsum = rts/np.arange(1,len(rts)+1)
    if predict_q_table:
        predictions = np.squeeze(q1(np.expand_dims(np.array(betas_test),axis=1)).numpy())
        history = []
        history_1 = []

        for b in betas_test:
            history.append([[b, 0.]])
            history_1.append([[b,1.]])
        pred_guess_0 = np.squeeze(q2(np.expand_dims(np.array(history), axis=1)).numpy())
        pred_guess_1 = np.squeeze(q2(np.expand_dims(np.array(history_1), axis=1)).numpy())
        pred_guess = [pred_guess_0, pred_guess_1]

    plot_evolution_vKennedy_last(rt=rtsum, pt=pt, optimal=optimal, betas=betas_test,
     preds=predictions , pred_guess = pred_guess, loss=loss_ev, history_betas=history_betas, history_would_have_done=history_would_have_done,
      run_id= run_id)
    #
    save_models(run_id, models=[q1,q2,actor])
    data = "buffer_size: {}\Batch size: {}\nLearning_rate: {}\nOptimizer Critic: {}\nOptimizer Actor: {}".format(str(buffer_size), str(batch_size), str(lr),optimizer_critic.__str__(),optimizer_actor.__str__()  )
    #
    os.chdir(run_id)
    with open("info.txt", 'w') as f:
        f.write(data)
        f.close()
    os.chdir("..")

    return q1, q2, actor


check_folder("mate")

run_id=record()
number_run = "run_"+str(run_id)
q1, q2, actor = real_training(run_id=number_run, lr=1e-2,
 states_wasted=10**2, batch_size=1000, buffer_size=2*10**3)


  2%|▏         | 2/100 [00:00<00:06, 14.09it/s]

******
******
[[0.51152975 0.50286901]
 [0.51154317 0.50281905]
 [0.51155659 0.50276908]
 [0.51157    0.50271912]
 [0.5115795  0.50266244]
 [0.51157539 0.50258238]
 [0.51157127 0.50250233]
 [0.51156172 0.50241532]
 [0.51152348 0.50229161]
 [0.51148524 0.50216789]]
[[0.5111035  0.50306193]
 [0.51157104 0.50249774]]
[0.5111034962782872, 0.5115710355942864]
          
chechehcehchehchhcehceh
[[0.50818617 0.49980666]
 [0.50821564 0.49971585]
 [0.50719308 0.50033007]]
[0.5081861698145073, 0.5082156400755188, 0.507193083804113]
          
chechehcehchehchhcehceh
[[0.51042977 0.50192093]
 [0.51041038 0.5019891 ]
 [0.50976029 0.50260108]
 [0.51040552 0.50200619]]
[0.5104297668830883, 0.5104103787040674, 0.5097602948543423, 0.5104055163891701]
          
chechehcehchehchhcehceh


  6%|▌         | 6/100 [00:00<00:06, 14.93it/s]

[[0.51301149 0.50422354]
 [0.51272574 0.50485898]
 [0.51300362 0.50424994]
 [0.51300075 0.50425491]
 [0.51269071 0.50491963]]
[0.5130114908131069, 0.5127257411019838, 0.513003619335974, 0.513000749606189, 0.5126907109706118]
          
chechehcehchehchhcehceh
[[0.51543545 0.50639663]
 [0.51543643 0.50641049]
 [0.51544264 0.50649839]
 [0.51543668 0.50641396]
 [0.5155264  0.50715967]
 [0.5155294  0.5072021 ]]
[0.515435451586522, 0.515436430335071, 0.5154426379273293, 0.5154366757928045, 0.5155264005817642, 0.515529396703288]
          
chechehcehchehchhcehceh
[[0.51793198 0.50884808]
 [0.51799646 0.50896876]
 [0.51792181 0.50882905]
 [0.51839281 0.50992804]
 [0.51836169 0.50986979]
 [0.51793453 0.50885285]
 [0.5179175  0.50882099]]
[0.5179319779215661, 0.5179964640212611, 0.5179218103743777, 0.5183928145315854, 0.5183616915639693, 0.5179345278114238, 0.5179175017489327]
          
chechehcehchehchhcehceh
[[0.52059476 0.51158669]
 [0.52058626 0.5115737 ]
 [0.52055573 0.51152711]
 [0.52058

 10%|█         | 10/100 [00:00<00:06, 14.43it/s]

...

[[0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]
 [0.50635827 0.50577196]


 96%|█████████▌| 96/100 [00:05<00:00, 17.04it/s]

[[0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]
 [0.50399735 0.50372957]


100%|██████████| 100/100 [00:06<00:00, 17.25it/s]

[[0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]
 [0.50354084 0.50346871]


In [ ]:
pred_guess_0